In [2]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import pandas as pd

def get_unique_values_for_module(df, module):
    return np.unique(df[module])


def plot_barplots_of_module_impact(modAlgo, dim):
    budgets =  [50*dim, 100*dim, 300*dim, 500*dim, 1000*dim, 1500*dim]
    modules = ['elitist', 'mirrored', 'base_sampler', 'weights_option', 'local_restart', 'step_size_adaptation'] if modAlgo == "modCMA" else ['mutation_reference', 'mutation_base', 'mutation_n_comps','use_archive','crossover','adaptation_method','lpsr']
    conf_num = 324 if modAlgo == "modCMA" else 576
    if modAlgo == "modCMA":
        df_grid = pd.read_csv('../data/raw_data/modCMA_conf_grid.csv', index_col = 0)
    elif modAlgo == "modDE":
        df_grid = pd.read_csv('../data/raw_data/modDE_conf_grid.csv', index_col = 0)
    df_grid = df_grid.replace({np.nan: 'None'})

    fig = make_subplots(rows=len(budgets), cols=len(modules), shared_xaxes=True, shared_yaxes=True, vertical_spacing=0.03, horizontal_spacing=0.01, subplot_titles=modules)
    colors = ['#337CA0', '#8FBC94', '#89AAD2', '#619b8a']
    row = 1
    for budget in budgets:
        col = 1
        for module in modules:
            module_options = get_unique_values_for_module(df_grid, module)
            data = [module_options]
            z = 0
            for option in module_options:
                confs = list(df_grid[df_grid[module]==option].index)
                TPs = []
                for conf_id in confs:
                    TP = np.mean(np.array(pd.read_csv(f"../data/performance_data/{modAlgo}/log/budget_{budget}_conf_{conf_id}_{dim}D.csv", index_col=0)))
                    TPs.append(TP)
              
                fig.add_trace(
                    # go.Bar(x=[str(option)], y=[np.mean(TPs)], name = str(option), marker={'color': colors[z]}), 
                    
                    go.Violin(y=TPs, box={'visible': True}, name=f"{option}", width = 0.95, marker={'color': colors[z]}), row=row, col=col)
                    # go.Box(y=TPs, name=f"{option}", marker={'color': colors[z]}), row=row, col=col)
                z += 1
            col += 1
        row += 1
    fig.update_traces({
        'showlegend':False
        })
    fig.update_layout(
        height=800,
        width = 1100,
        margin=dict(
                l=0,
                r=0,
                b=0,
                t=30,
                ),
        font = dict(
                    size=18
                ),
    )
    fig.update_xaxes(tickangle=35)
    fig['layout']['yaxis1']['title']= '50D'
    fig['layout']['yaxis'+str(1+1*len(modules))]['title']= '100D'
    fig['layout']['yaxis'+str(1+2*len(modules))]['title']= '300D'
    fig['layout']['yaxis'+str(1+3*len(modules))]['title']= '500D'
    fig['layout']['yaxis'+str(1+4*len(modules))]['title']= '1000D'
    fig['layout']['yaxis'+str(1+5*len(modules))]['title']= '1500D'
    fig.show()
    fig.write_image(f"../figures/module_impact_violin_plots/{modAlgo}_{dim}.pdf")

# plot_barplots_of_module_impact('modCMA', 5)
# plot_barplots_of_module_impact('modCMA', 30)
plot_barplots_of_module_impact('modDE', 5)
plot_barplots_of_module_impact('modDE', 30)